In [1]:
import pyodbc
import numpy as np
import pandas as pd

In [2]:
sql_driver = 'DRIVER={ODBC Driver 13 for SQL Server};'
sql_server = 'SERVER=sage;'
sql_db = 'DATABASE=BUYS;'
sql_UID = 'Trusted_Connection=yes;'

cnxn = pyodbc.connect(sql_driver + sql_server + sql_db + sql_UID)

def fetch_data(q, cnxn):
    df = pd.read_sql(sql=q, con=cnxn)
    return df

In [3]:
query_PL = '''
SET NOCOUNT ON
SET ANSI_WARNINGS OFF

DECLARE @StartDate DATE = '1/1/2010'

SELECT 
	[Date] [BusinessMonth],
	COUNT(DISTINCT p.Loc) [count_Locations],
	SUM(Income) [total_Income],
	SUM(PurchasesCash) [total_PurchasesCash],
	SUM([Purchases-Other]) [total_PurchasesOther],
	SUM(CostOfGoodsSold) [total_CostOfGoodsSold],
	SUM(GrossProfit) [total_GrossProfit],
	SUM(NetIncomeLoss) [total_NetIncomeLoss],
	SUM(Commissions) [total_Commissions]
INTO #PL
FROM ReportsView..pnl p
	INNER JOIN ReportsView..StoreLocationMaster slm
		ON p.Loc = slm.LocationNo 
		AND slm.StoreStatus = 'O'
		AND slm.OpenDate <= @StartDate
		AND slm.StoreType = 'S'
WHERE [Date] >= @StartDate
GROUP BY [Date]

SELECT 
	DATEADD(MONTH, DATEDIFF(MONTH, 0, bbh.StartDate), 0) [BusinessMonth],
	COUNT(DISTINCT bbh.LocationID) [count_Locations],
	COUNT(bbh.BuyXactionID) [count_BuyTransactions],
	SUM(bbh.TotalQuantity) [total_UsedPurchaseQty],
	SUM(bbh.TotalOffer) [total_UsedPurchaseAmt],
	bih.total_UsedPurchaseSipsQty
INTO #BuyTransactions
FROM rHPB_Historical..BuyHeaderHistory bbh
	INNER JOIN ReportsView..StoreLocationMaster slm
		ON bbh.LocationID = slm.LocationID
		AND slm.StoreStatus = 'O'
		AND slm.OpenDate <= @StartDate
		AND slm.StoreType = 'S'
	LEFT OUTER JOIN (
		SELECT
			DATEADD(MONTH, DATEDIFF(MONTH, 0, bbh.StartDate), 0) [BusinessMonth],
			SUM(CASE
				WHEN bih.BuyType NOT IN ('CSU', 'CX', 'MG', 'VDU')
				THEN bih.Quantity
				END) [total_UsedPurchaseSipsQty]
		FROM rHPB_Historical..BuyItemHistory bih
			INNER JOIN rHPB_Historical..BuyHeaderHistory bbh
				ON bbh.BuyXactionID = bih.BuyXactionID
				AND bbh.LocationID = bih.LocationID
			INNER JOIN ReportsView..StoreLocationMaster slm
				ON bih.LocationID = slm.LocationID
				AND slm.StoreStatus = 'O'
				AND slm.OpenDate <= @StartDate
				AND slm.StoreType = 'S' 
		GROUP BY DATEADD(MONTH, DATEDIFF(MONTH, 0, bbh.StartDate), 0)) bih
			ON bih.BusinessMonth = DATEADD(MONTH, DATEDIFF(MONTH, 0, bbh.StartDate), 0)
		
WHERE 
	bbh.BusinessDate >= @StartDate AND
	bbh.Status = 'A'
GROUP BY DATEADD(MONTH, DATEDIFF(MONTH, 0, bbh.StartDate), 0), bih.total_UsedPurchaseSipsQty

SELECT 
	DATEADD(MONTH, DATEDIFF(MONTH, 0, spi.DateInStock), 0) [BusinessMonth],
	COUNT(spi.ItemCode) [total_SipsItemsQty]
INTO #SipsItems
FROM ReportsData..SipsProductInventory spi
GROUP BY DATEADD(MONTH, DATEDIFF(MONTH, 0, spi.DateInStock), 0)

SELECT 
	DATEADD(MONTH, DATEDIFF(MONTH, 0, sr.ProcessDate), 0) [BusinessMonth],
	SUM(sr.Qty) [total_NewReceivedQty]
INTO #DistributionShipments
FROM (
	SELECT 
		sr.ProcessDate,
		sr.Qty
	FROM ReportsView..vw_StoreReceiving sr
	WHERE 
		sr.ShipmentType  IN ('W', 'R') AND
		sr.ProcessDate >= @StartDate
	UNION ALL
	SELECT 
		srh.ProcessDate,
		srh.Qty
	FROM ReportsView..vw_StoreReceiving_Historical srh
	WHERE 
		srh.ShipmentType  IN ('W', 'R') AND
		srh.ProcessDate >= @StartDate
	) sr
GROUP BY DATEADD(MONTH, DATEDIFF(MONTH, 0, sr.ProcessDate), 0)


SELECT 
	DATEADD(MONTH, DATEDIFF(MONTH, 0, shh.StartDate), 0) [BusinessMonth],
	COUNT(shh.SalesXactionID) [count_SalesTransactions],
	SUM(shh.TotalDue) - SUM(shh.SalesTax) [total_SalesAmount]
INTO #SalesTransactions
FROM rHPB_Historical..SalesHeaderHistory shh
	INNER JOIN ReportsView..StoreLocationMaster slm
		ON shh.LocationID = slm.LocationID
		AND slm.StoreStatus = 'O'
		AND slm.OpenDate <= @StartDate
		AND slm.StoreType = 'S'
WHERE 
	shh.Status = 'A'  AND 
	shh.StartDate > @StartDate
GROUP BY DATEADD(MONTH, DATEDIFF(MONTH, 0, shh.StartDate), 0)
ORDER BY BusinessMonth

SELECT 
	pl.BusinessMonth,
	pl.total_Income,
	pl.total_PurchasesCash,
	pl.total_PurchasesOther,
	pl.total_CostOfGoodsSold,
	pl.total_GrossProfit,
	pl.total_NetIncomeLoss,
    pl.total_Commissions,
	pl.count_Locations,
	bt.count_Locations,
	ds.total_NewReceivedQty,
	bt.count_BuyTransactions,
	bt.total_UsedPurchaseQty,
	bt.total_UsedPurchaseSipsQty,
	bt.total_UsedPurchaseAmt,
	si.total_SipsItemsQty,
	bt.total_UsedPurchaseAmt / bt.total_UsedPurchaseQty [avg_ItemCost],
	st.count_SalesTransactions,
	st.total_SalesAmount
FROM #PL pl
INNER JOIN #BuyTransactions bt
	ON pl.BusinessMonth = bt.BusinessMonth
INNER JOIN #SipsItems si
	ON pl.BusinessMonth = si.BusinessMonth
INNER JOIN #SalesTransactions st
	ON pl.BusinessMonth = st.BusinessMonth
INNER JOIN #DistributionShipments ds
	ON pl.BusinessMonth = ds.BusinessMonth
ORDER BY BusinessMonth


DROP TABLE #PL
DROP TABLE #BuyTransactions
DROP TABLE #SipsItems
DROP TABLE #SalesTransactions
DROP TABLE #DistributionShipments

'''

query_Inv = '''
SET NOCOUNT ON
SET ANSI_WARNINGS OFF

SELECT DISTINCT
	sir.Inventory_ID,
	sir.Inventory_Description,
	CASE 
		WHEN sir.Inventory_Description LIKE '%Jan%'
			THEN 1
		WHEN sir.Inventory_Description LIKE '%Jun%'
			THEN 6
		END [Month],
	DATEPART(YEAR, EndDate) [Year]
INTO #InventoryIndex
FROM HPB_INV..Scheduled_Inventory_Reporting sir
WHERE 
	sir.Inventory_Description LIKE '%Scheduled Inventory%'
	AND sir.Inventory_Description NOT LIKE '%Pilot%'
ORDER BY [Year], [Month]

SELECT  
	DATEFROMPARTS(ii.Year, ii.Month, 1) [InventoryDate],
	RTRIM(LTRIM(ProductType)) [ProductType],
	CASE WHEN sir.Inventory_SectionID = 98 THEN 'Not On Sales Floor' ELSE 'Sales Floor' END [SalesFloor],
	CASE WHEN sir.ItemType_Description = 'Distribution' THEN 'New' ELSE 'Used' END [ItemClass],
	SUM(sir.Quantity * sir.Factor) [total_Quantity],
	SUM(sir.Cost * sir.Factor) [total_Cost],
	SUM(sir.Price) [total_Price_NoFactor],
	SUM(sir.Quantity) [total_Quantity_NoFactor]
INTO #UngroupedSummary
FROM HPB_INV..Scheduled_Inventory_Reporting sir
	INNER JOIN #InventoryIndex ii
		ON sir.Inventory_ID = ii.Inventory_ID
	INNER JOIN ReportsView..StoreLocationMaster slm
		ON sir.LocationID = slm.LocationId
		AND slm.OpenDate < '1/1/2015'
		AND slm.StoreStatus = 'O'
		AND slm.StoreType = 'S'
		
WHERE sir.Price < 100000
GROUP BY DATEFROMPARTS(ii.Year, ii.Month, 1), sir.ProductType, sir.Inventory_SectionID, sir.ItemType_Description


SELECT 
	us.InventoryDate,
	us.ItemClass,
	us.ProductType,
    pt.PTypeGroup,
	us.SalesFloor,
	SUM(us.total_Quantity) [total_Quantity],
	SUM(us.total_Cost) [total_Cost],
	SUM(us.total_Cost) / SUM(us.total_Quantity) [avg_Cost],
	SUM(us.total_Price_NoFactor) / SUM(us.total_Quantity_NoFactor) [avg_Price]
FROM #UngroupedSummary us
    INNER JOIN ReportsData..ProductTypes pt
        ON LTRIM(RTRIM(us.ProductType)) = LTRIM(RTRIM(pt.ProductType))
GROUP BY us.InventoryDate, us.ItemClass, us.ProductType, pt.PTypeGroup, us.SalesFloor
ORDER BY ItemClass DESC, ProductType, SalesFloor DESC, InventoryDate




DROP TABLE #InventoryIndex
DROP TABLE #UngroupedSummary
'''

In [4]:
query_LocInv = '''
SET NOCOUNT ON
SET ANSI_WARNINGS OFF

SELECT DISTINCT
	sir.LocationNo,
	sir.Inventory_ID,
	sir.Inventory_Description,
	CASE 
		WHEN sir.Inventory_Description LIKE '%Jan%'
			THEN 1
		WHEN sir.Inventory_Description LIKE '%Jun%'
			THEN 6
		END [Month],
	DATEPART(YEAR, EndDate) [Year]
INTO #InventoryIndex
FROM HPB_INV..Scheduled_Inventory_Reporting sir
WHERE 
	sir.Inventory_Description LIKE '%Scheduled Inventory%'
	AND sir.Inventory_Description NOT LIKE '%Pilot%'
ORDER BY [Year], [Month]

SELECT
	sir.LocationNo,
	DATEFROMPARTS(ii.Year, ii.Month, 1) [InventoryDate],
	SUM(sir.Quantity * sir.Factor) [total_Quantity],
	SUM(sir.Cost * sir.Factor) [total_Cost],
	SUM(sir.Price) [total_Price_NoFactor],
	SUM(sir.Quantity) [total_Quantity_NoFactor],
	SUM(sir.Price)/SUM(sir.Quantity) [avg_Price]
INTO #SalesFloorTotals
FROM HPB_INV..Scheduled_Inventory_Reporting sir
	INNER JOIN #InventoryIndex ii
		ON sir.Inventory_ID = ii.Inventory_ID
	INNER JOIN ReportsView..StoreLocationMaster slm
		ON sir.LocationID = slm.LocationId
		AND slm.OpenDate < '1/1/2015'
		AND slm.StoreStatus = 'O'
		AND slm.StoreType = 'S'
WHERE 
	sir.Price < 1000000 AND
	sir.Inventory_SectionID <> 98
GROUP BY sir.LocationNo, DATEFROMPARTS(ii.Year, ii.Month, 1) 

SELECT  
	sir.LocationNo,
	DATEFROMPARTS(ii.Year, ii.Month, 1) [InventoryDate],
	RTRIM(LTRIM(ProductType)) [ProductType],
	CASE WHEN sir.Inventory_SectionID = 98 THEN 'Not On Sales Floor' ELSE 'Sales Floor' END [SalesFloor],
	CASE WHEN sir.ItemType_Description = 'Distribution' THEN 'New' ELSE 'Used' END [ItemClass],
	SUM(sir.Quantity * sir.Factor) [total_Quantity],
	SUM(sir.Cost * sir.Factor) [total_Cost],
	SUM(sir.Price) [total_Price_NoFactor],
	SUM(sir.Quantity) [total_Quantity_NoFactor]
INTO #UngroupedSummary
FROM HPB_INV..Scheduled_Inventory_Reporting sir
	INNER JOIN #InventoryIndex ii
		ON sir.Inventory_ID = ii.Inventory_ID
	INNER JOIN ReportsView..StoreLocationMaster slm
		ON sir.LocationID = slm.LocationId
		AND slm.OpenDate < '1/1/2015'
		AND slm.StoreStatus = 'O'
		AND slm.StoreType = 'S'
WHERE sir.Price < 100000
GROUP BY sir.LocationNo, DATEFROMPARTS(ii.Year, ii.Month, 1), sir.ProductType, sir.Inventory_SectionID, sir.ItemType_Description


SELECT 
	us.InventoryDate,
	us.LocationNo,
	us.ItemClass,
	us.ProductType,
    pt.PTypeGroup,
	us.SalesFloor,
	SUM(us.total_Quantity) [total_Quantity],
	CAST(SUM(us.total_Quantity) AS FLOAT)/ 
		NULLIF(CAST(sft.total_Quantity AS FLOAT), 0) [pct_QuantityOnSalesFloor],
	SUM(us.total_Cost) [total_Cost],
	CAST(SUM(us.total_Cost) AS FLOAT)/ 
		NULLIF(CAST(sft.total_Cost AS FLOAT), 0) [pct_CostOnSalesFloor],
	SUM(us.total_Cost) / NULLIF(SUM(us.total_Quantity), 0) [avg_Cost],
	SUM(us.total_Price_NoFactor) / NULLIF(SUM(us.total_Quantity_NoFactor), 0) [avg_Price],
	CAST(SUM(us.total_Price_NoFactor) AS FLOAT)/ 
		NULLIF(CAST(sft.total_Price_NoFactor AS FLOAT), 0) [pct_PriceOnSalesFloor]
FROM #UngroupedSummary us
    INNER JOIN ReportsData..ProductTypes pt
        ON LTRIM(RTRIM(us.ProductType)) = LTRIM(RTRIM(pt.ProductType))
	INNER JOIN #SalesFloorTotals sft
		ON us.LocationNo = sft.LocationNo AND
		us.InventoryDate = sft.InventoryDate
GROUP BY us.InventoryDate, us.LocationNo, us.ItemClass, us.ProductType, pt.PTypeGroup, us.SalesFloor, sft.total_Quantity, sft.total_Price_NoFactor, sft.total_Cost
ORDER BY ItemClass DESC, ProductType, SalesFloor DESC, InventoryDate


DROP TABLE #InventoryIndex
DROP TABLE #SalesFloorTotals
DROP TABLE #UngroupedSummary
'''

query_LocPL = '''
SET NOCOUNT ON
SET ANSI_WARNINGS OFF

SELECT 
	[Date],
	slm.LocationNo [LocationNo],
    slm.SalesFloorSize,
	SUM(Income) [total_Income],
    SUM(Income)/NULLIF(slm.SalesFloorSize, 0) [total_IncomePerSqFt],
	SUM(PurchasesCash) [total_PurchasesCash],
	SUM([Purchases-Other]) [total_PurchasesOther],
	SUM(CostOfGoodsSold) [total_CostOfGoodsSold],
	SUM(GrossProfit) [total_GrossProfit],
    SUM(GrossProfit)/NULLIF(slm.SalesFloorSize, 0) [total_GrossProfitPerSqFt],
	SUM(NetIncomeLoss) [total_NetIncomeLoss]
FROM ReportsView..pnl p
	INNER JOIN ReportsView..StoreLocationMaster slm
		ON p.Loc = slm.LocationNo 
		AND slm.StoreStatus = 'O'
		AND slm.StoreType = 'S'
WHERE [Date] >= '1/1/2009'
GROUP BY [Date], slm.LocationNo, slm.SalesFloorSize
ORDER BY [Date], slm.LocationNo
'''

In [5]:
df_PL  = fetch_data(query_PL, cnxn)
df_Inv  = fetch_data(query_Inv, cnxn)
df_LocInv = fetch_data(query_LocInv, cnxn)
df_LocPL = fetch_data(query_LocPL, cnxn)

In [6]:
df_PL.to_csv('./PLAnalysis.csv')
df_Inv.to_csv('./InvAnalysis.csv')
df_LocPL.to_csv('./LocPLAnalysis.csv')
df_LocInv.to_csv('./LocInvAnalysis.csv')

In [7]:
cnxn.close()